In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import math

In [ ]:
def func_power(f, x, exp):
    ret = x
    for _ in range(exp):
        ret = f(ret)
    return ret

def steps_to_target(f, x, target):
    steps = 0
    curr_val = x
    while curr_val != target:
        curr_val = f(curr_val)
        steps += 1
    return steps

In [ ]:
data = np.array([
    [steps_to_target(
        lambda x: math.floor(math.sqrt(x)),
        x,
        1
    ),
    steps_to_target(
        lambda x: x // 2,
        x,
        0
    ),
    steps_to_target(
        lambda x: x // 4,
        x,
        0
    ),
    steps_to_target(
        lambda x: x // 8,
        x,
        0
    ),
    steps_to_target(
        lambda x: math.floor(math.log2(x)),
        x,
        0
    ),
    steps_to_target(
        lambda x: math.floor(math.log10(x)),
        x,
        0
    )
    ] 
for x in range(1, 1_000_000)])

df = pd.DataFrame(data=data, columns=["sqrt", "div2", "div4", "div8", "log2", "log10"])

fig = px.line(df, labels={"index": "Number", "value": "Steps to zero"})
fig.show() 